# Rebalanceamento de Carteira:

### Import de Bibliotecas:

In [1]:
import re
import locale
import numpy as np
import pandas as pd
from skimage.measure import label, regionprops

locale.setlocale(locale.LC_ALL, 'pt-BR.utf-8')

'pt-BR.utf-8'

### Leitura dos Excels:

In [2]:
path = './Dados'

In [3]:
carteira = pd.read_excel(f'{path}/Portfolio Global - KING.xlsx', sheet_name='Carteira Global', header=None)
politica = pd.read_excel(f'{path}/Portfolio Global - KING.xlsx', sheet_name='Política de Investimento', header=None)
database = pd.read_excel(f'{path}/Planilha PFE Classificadora.xlsx', sheet_name='result', header=None)

### Extração de Tabelas e Atribuição de Variáveis Dataframe

#### Análise de Tabelas dentro de cada Excel:

In [4]:
def getTables(df):
    larr = label(np.array(df.notnull()).astype("int"))
    
    list_dfs = []
    
    for s in regionprops(larr):
        sub_df = df.iloc[
            s.bbox[0]:s.bbox[2], s.bbox[1]:s.bbox[3]
        ].pipe(
            lambda df_: df_.rename(columns=df_.iloc[0]).drop(df_.index[0])
        )
    
        list_dfs.append(sub_df)
    return list_dfs


In [5]:
dfsCarteira = getTables(carteira)
dfsPolitica = getTables(politica)
dfsDatabase = getTables(database)

#### Criando Variáveis para cada Tabela dos Excels

In [6]:
def attrDataframes(list_dfs):
    def getVarName(df):
        def namestr(obj, namespace):
            return [name for name in namespace if namespace[name] is obj]
    
        return namestr(df, globals())[0]
    
    for i in range(len(list_dfs)):
        dfName = getVarName(list_dfs).replace('dfs', '')
        
        varName = f'df{dfName}{i+1}'
        value = list_dfs[i]
        globals()[varName] = value
        
        print(f'- Dataframe {varName} created.')
    print('')


In [7]:
attrDataframes(dfsCarteira)
attrDataframes(dfsPolitica)
attrDataframes(dfsDatabase)

- Dataframe dfCarteira1 created.

- Dataframe dfPolitica1 created.
- Dataframe dfPolitica2 created.
- Dataframe dfPolitica3 created.

- Dataframe dfDatabase1 created.



### Dataframes:

#### - Carteira:

In [8]:
valorCarteira = dfCarteira1["Financeiro"].iloc[-1]
perctCarteira = dfCarteira1["%"].iloc[-1]

In [9]:
dfCarteira1 = dfCarteira1.dropna(subset=["Source"])
dfCarteira1 = dfCarteira1.dropna(subset=["Portfólio"])

In [10]:
dfCarteira1.head()

,Data Posição,Source,Portfólio,Quantidade,Financeiro,Classe,%
5,2023-09-13 00:00:00,KING - BRL,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668
9,2023-09-13 00:00:00,KING KONG FIM CP IE,LFT01092026,38,264782.350653,Renda Fixa Brasil Pós Fixado,0.021534


#### - Política:

In [11]:
dfPolitica1.head()

,TradingDesk,Book,Min,Max,Estratégico,Tático,Atual
1,KING - BRL,Renda Fixa BR Pós-Fixado,0.05,0.35,0.075,0.13,0.233557
2,KING - BRL,Renda Fixa BR Crédito Privado,0,0.2,0.09,0.09,0.100853
3,KING - BRL,Renda Fixa BR Pré-Fixado,0,0.1,0.04,0,0
4,KING - BRL,Renda Fixa BR Inflação,0,0.15,0.08,0.12,0.109807
5,KING - BRL,Renda Fixa Intl Cash Equivalent,0,0.05,0,0.005,0.004163


In [12]:
dfPolitica2.rename(columns={'%':'% Target'}, inplace=True)
dfPolitica2.head()

,Alocação Estratégica por Mercado,% Target
23,Renda Fixa,0.315
24,Retorno Absoluto,0.15
25,Renda Variável,0.225
26,Private Equity,0.255
27,Real Estate,0.055


In [13]:
dfPolitica3.head()

,Alocação Estratégica por Geografia,%
31,Brasil,0.575
32,Internacional,0.425


#### - Database:

In [14]:
dfDatabase1.head()

,AssetClassName,AssetClassLevelTwoName,Benchmark,AssetClassLevelThreeName,AssetName,AssetInvestmentTypeName,AssetRedemptionGate,AssetRedemptionLockUpDayCountName,AssetRedemptionLockUpDayCountValue,AssetRedemptionMonths,AssetRedemptionQuotationDelayDayCountName,AssetRedemptionQuotationDelayDayCountValue,AssetRedemptionQuotationFrequencyName,AssetRedemptionSettlementDelayDayCountName,AssetRedemptionSettlementDelayDayCountValue,AssetIssueMonths,AssetIssueDelayDayCountName,AssetIssueDelayDayCountValue,AssetIssueQuotationFrequencyName
2,Private Equity,Private Equity Internacional,S&P500,Private Equity Internacional Multiestratégia,Jera - JWM Illiquid SP III- I-U- Initial,InvestmentFundOffShore,100.0,Meses,60.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,30.0,Primeiro dia do Mês,Dias Corridos,30.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,0,Primeiro dia do Mês
3,Caixa / Conta Corrente / Provisões,Caixa / Conta Corrente / Provisões,CDI,Caixa / Conta Corrente / Provisões,Empréstimo USD Tapioca,Provisions and Costs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Private Equity,Private Equity Internacional,S&P500,Private Equity Internacional Venture Capital,Coller International Partners VIII Feeder Fund,InvestmentFundOffShore,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia,Dias Úteis,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia
5,Renda Fixa Brasil Crédito Pós-Fixado,RF BR Crédito Pós-Fixado Crédito Estruturado,CDI,NaN,JWM Crédito FIM CP,InvestmentFund,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,180.0,Fechamento do Dia,Dias Corridos,1.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia
6,Renda Fixa,Renda Fixa Brasil Inflação,CDI,RF BR Inflação Soberano,JWM Inflação FIM CP,InvestmentFund,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,7.0,Fechamento do Dia,Dias Úteis,1.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia


### Rebalanceamento

#### Pegando Ativos da Carteira

In [15]:
result_list = dfCarteira1['Portfólio'].dropna().to_list()

result_list

['BV CASH FIRF SIMPLES',
 'BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF',
 'LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO',
 'LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO',
 'LFT01092026',
 'TESOURO SELIC FI RF',
 'JWM CREDITO FIM CP',
 'JWM INFLAÇÃO FIM CP',
 'JWM MULTIMERCADO FIM IE',
 'BOVA11',
 'JWM AÇÕES FIA',
 'MAESTRO FIA',
 'IVVB11',
 'NOVA RAPOSO FIP',
 'JWM VANQUISH FIM CP IE',
 'SPECTRA NIMBUS FIP',
 'JWM ILLIQUID SP III',
 'GTWR11 - FII G TOWERS CI',
 'FII VOTORANTIM LECREC',
 'JWM LIQUID SP II',
 'WDOV23']

In [16]:
for i in dfDatabase1["Benchmark"].unique():
    print(i)

S&P500
CDI
IBOVESPA
IFIX
ICB
HASH11
DOLAR


In [17]:
filterWords = {}

for benchmark in dfDatabase1["Benchmark"].unique():
    dfFiltrado = dfDatabase1[dfDatabase1["Benchmark"] == benchmark][["AssetClassName", "AssetName", "Benchmark"]]
    
    dfFiltrado["AssetClassName"] = dfFiltrado["AssetClassName"].astype(str).str.lower().apply(lambda x: re.sub(r'\d+', '', x))
    dfFiltrado["AssetName"] = dfFiltrado["AssetName"].astype(str).str.lower().apply(lambda x: re.sub(r'\d+', '', x))
    
    colunas = dfFiltrado[["AssetClassName", "AssetName"]]
    words = ' '.join(colunas.values.flatten()).split()
    words = list(set(words))

    filterWords[benchmark] = words

# filterWords

In [18]:
# Create a set for each list
list_sets = {key: set(value) for key, value in filterWords.items()}

# Find unique words in each list that do not appear in other lists
unique_words_dict = {key: list(word_set - set.union(*(list_sets[k] for k in list_sets if k != key))) for key, word_set in list_sets.items()}

# Print the result
for key, unique_words in unique_words_dict.items():
    print(f'{key}: {unique_words}\n')

S&P500: ['cn', 'bain', 'iu-', 'índice', 'heptagon', 'i-u-', 'xcala', 'jera', 'l-u-', 'morgan', 'gps', 'international', 'investimento', 'ii,', 'south', 'investors', 'wcm', 'sts', 'no', 'viii', 'balanceados', 'l.p.', 'coller', 'ivvb', 'gcm', 'fundo', 'liquid', 'western', 'star', 'illiquid', 'technology', 'boom', 'cdr', 'exterior', 'battery', 'grosvenor', 'bw', 'aaa', 'cotas', 'parallax', 'spv', 'metals', 'bc', 'world', 'sp', 'j.p.', 'growth', 'em', 'supersonic', 'iwda', 'offshore-', 'initial', 'iii-', 'onevalley', 'ii-']

CDI: ['pré-fixado', 'hapvida', ',.', '-yr', 'despesa', 'itsa', 'tesouro', 'bond', 'green', 'eur', 'klabin', 'di', 'gbp', 'outros_', 'provisões', 'digimais', 'lf', 'corretagem', 'fidis', 'jdc', '+', 'américa', 'npl', 'original', 'aggu', 'interest', 'ahl', 'safra', 'inflação', 'usd_', 'fiji', 'ajuste', 'funds', 'mult', 'seleção', 'provision', 'chimera', 'pimco', '%', 'sps', '.', 'auditoria_', 'recr', 'term', 'year', 'plc', 'despesas', 'pós-fixado', 'crédito', 'cemig', 'tr

#### Categorizando cada Ativo com seu Benchmark

In [19]:
'''
Dúvidas:
FII = IFIX ou CDI
FIP = IBOVESPA OU IFIX
NOVA RAPOSA FIP = IBOV
CASH ou SIMPLES = CDI
LECREC = IFIX
'''

# CDI
# substringsCDI = ['tesouro', 'lci', 'lft', 'lca', 'credito fim cp']
substringsCDI = unique_words_dict["CDI"]
patternsCDI = '|'.join(substringsCDI)

# DOLAR
# substringsDOLAR = []
substringsDOLAR = unique_words_dict["DOLAR"]
patternsDOLAR = '|'.join(substringsDOLAR)

# HASH11
# substringsHASH = []
substringsHASH = unique_words_dict["HASH11"]
patternsHASH = '|'.join(substringsHASH)

# IBOVESPA
# substringsIBOV = ['fia']
substringsIBOV = unique_words_dict["IBOVESPA"]
patternsIBOV = '|'.join(substringsIBOV)

# ICB
# substringsICB = []
substringsICB = unique_words_dict["ICB"]
patternsICB = '|'.join(substringsICB)

# IFIX
# substringsIFIX = []
substringsIFIX = unique_words_dict["IFIX"]
patternsIFIX = '|'.join(substringsIFIX)

# S&P500
# substringsSP500 = ['illiquid', 'liquid']
substringsSP500 = unique_words_dict["S&P500"]
patternsSP500 = '|'.join(substringsSP500)


In [20]:
def matchFund(target, filter):
    return any(substring in target.split() for substring in filter)

In [35]:
cont = 0
benchmarks = []

for i in result_list:
    source = dfDatabase1['AssetName'].astype(str).str.lower().apply(lambda x: re.sub(r'\d+', '', x))
    search  = re.sub(r'\d+', '', i.lower())
    
    result = dfDatabase1[source == search][['Benchmark', 'AssetName']]
    
    if not result.empty:
        cont += 1
        benchmarks.append(result['Benchmark'].iloc[0])
        # print(result['Benchmark'].iloc[0])
        
    elif matchFund(search, substringsCDI):
        cont+=1
        benchmarks.append('CDI')
        # print('CDI')
        
    elif matchFund(search, substringsDOLAR):
        cont+=1
        benchmarks.append('DOLAR')
        # print('DOLAR')
        
    elif matchFund(search, substringsHASH):
        cont+=1
        benchmarks.append('HASH11')
        # print('HASH11')
        
    elif matchFund(search, substringsIBOV):
        cont+=1
        benchmarks.append('IBOVESPA')
        # print('IBOVESPA')
        
    elif matchFund(search, substringsICB):
        cont+=1
        benchmarks.append('ICB')
        # print('ICB')
        
    elif matchFund(search, substringsIFIX):
        cont+=1
        benchmarks.append('IFIX')
        # print('IFIX')
        
    elif matchFund(search, substringsSP500):
        cont+=1
        benchmarks.append('S&P500')
        # print('S&P500')
        
    else:
        benchmarks.append(np.nan)
        print(f"NÃO PASSEI | {search}")

    

print(f"{cont/len(result_list):.2%}")
# benchmarks

NÃO PASSEI | jwm credito fim cp
NÃO PASSEI | gtwr - fii g towers ci
NÃO PASSEI | wdov
85.71%


In [22]:
try:
    dfCarteira1.insert(2, 'Benchmarks', benchmarks)
except:
    pass

dfCarteira1

,Data Posição,Source,Benchmarks,Portfólio,Quantidade,Financeiro,Classe,%
5,2023-09-13 00:00:00,KING - BRL,CDI,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,CDI,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,CDI,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,CDI,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668
9,2023-09-13 00:00:00,KING KONG FIM CP IE,CDI,LFT01092026,38,264782.350653,Renda Fixa Brasil Pós Fixado,0.021534
10,2023-09-13 00:00:00,KING KONG FIM CP IE,CDI,TESOURO SELIC FI RF,219358.136714,440056.279454,Renda Fixa Brasil Pós Fixado,0.035789
12,2023-09-13 00:00:00,KING KONG FIM CP IE,NaN,JWM CREDITO FIM CP,14428.4466,1053194.94593,Renda Fixa Brasil Crédito Pós Fixado,0.085655
14,2023-09-13 00:00:00,KING KONG FIM CP IE,CDI,JWM INFLAÇÃO FIM CP,18400.612101,1350165.447481,Renda Fixa Brasil Inflação,0.109807
16,2023-09-13 00:00:00,KING KONG FIM CP IE,IBOVESPA,JWM MULTIMERCADO FIM IE,1659744.235928,1078016.393523,Retorno Absoluto Doméstico,0.087673
18,2023-09-13 00:00:00,KING KONG FIM CP IE,IBOVESPA,BOVA11,3499,202712.328683,Renda Variável Doméstico,0.016486


#### Agrupando os Benchmarks

In [23]:
groupedBenchmarks = dfCarteira1.groupby('Benchmarks')['%'].sum().reset_index()
groupedBenchmarks = groupedBenchmarks.rename(columns={'%':'% Atual'})

groupedBenchmarks

,Benchmarks,% Atual
0,CDI,0.343363
1,IBOVESPA,0.359357
2,IFIX,0.000196
3,S&P500,0.201034


#### Comparando Atual x Target

In [24]:
dicionario = {
    'CDI':'Renda Fixa',
    'IBOVESPA':'Retorno Absoluto',
    'S&P500':'Renda Variável',
    'IFIX':'Private Equity',
    'IFIX':'Real Estate',
    'ICB':'Commodities'
}

groupedBenchmarks['Benchmarks'] = groupedBenchmarks['Benchmarks'].map(dicionario)

groupedBenchmarks

,Benchmarks,% Atual
0,Renda Fixa,0.343363
1,Retorno Absoluto,0.359357
2,Real Estate,0.000196
3,Renda Variável,0.201034


In [25]:
dfRebalanced = dfPolitica2.merge(groupedBenchmarks, how='left', left_on='Alocação Estratégica por Mercado', right_on='Benchmarks')
dfRebalanced['% Atual'].fillna(0, inplace=True)
dfRebalanced.drop(columns=['Benchmarks'], inplace=True)
dfRebalanced['Offset'] = dfRebalanced['% Target'] - dfRebalanced['% Atual']

dfRebalanced

,Alocação Estratégica por Mercado,% Target,% Atual,Offset
0,Renda Fixa,0.315,0.343363,-0.028363
1,Retorno Absoluto,0.15,0.359357,-0.209357
2,Renda Variável,0.225,0.201034,0.023966
3,Private Equity,0.255,0.000000,0.255
4,Real Estate,0.055,0.000196,0.054804
5,Commodities,0,0.000000,0.0


#### Tracking Error:

In [26]:
# # ADAPTAR PARA OS FUNDOS

# import numpy as np
# import yfinance as yf
# from datetime import datetime, timedelta

# def get_returns(ticker, period):
#     start_date = (datetime.today() - timedelta(days=period)).strftime('%Y-%m-%d')
#     end_date = datetime.today().strftime('%Y-%m-%d')
#     data = yf.download(ticker, start=start_date, end=end_date)

#     data['Return'] = data['Adj Close'].pct_change()

#     return data['Return']


# def calculateTrackingError(stock, benchmark='^GSPC', period=30):
#     stockReturns = get_returns(stock, period)
#     benchmarkReturns = get_returns(benchmark, period)

#     excessReturns = stockReturns - benchmarkReturns

#     trackingError = np.std(excessReturns.dropna())

#     return trackingError


# stock = 'AAPL'
# # benchmark = '^GSPC'
# period = 30

# te = calculateTrackingError(stock, period=period)
# print(f"The Tracking Error is {te:.2%}")

In [27]:
dfRebalanced['Need Rebalance'] = [True, True, False, True, False, False]

dfRebalanced

,Alocação Estratégica por Mercado,% Target,% Atual,Offset,Need Rebalance
0,Renda Fixa,0.315,0.343363,-0.028363,True
1,Retorno Absoluto,0.15,0.359357,-0.209357,True
2,Renda Variável,0.225,0.201034,0.023966,False
3,Private Equity,0.255,0.000000,0.255,True
4,Real Estate,0.055,0.000196,0.054804,False
5,Commodities,0,0.000000,0.0,False


### Resultado Final:

In [28]:
if dfRebalanced['Need Rebalance'].any():
    pValue = locale.format_string("%.3f", valorCarteira, grouping=True, monetary=True)
    print(f"[Portfolio Value]\n- {pValue}\n")
    
    for i in range(len(dfRebalanced.columns) + 1):
        if dfRebalanced['Offset'][i] > 0:
            acao = 'Buy' 
        elif dfRebalanced['Offset'][i] <  0:
            acao = 'Sell'
        else:
            acao = 'Skip'

        price = abs(valorCarteira * dfRebalanced['Offset'][i])
        price = locale.format_string("%.3f", price, grouping=True, monetary=True)
        print(
f'''[{dfRebalanced['Alocação Estratégica por Mercado'][i]}]
- {acao} {price} of Porfolio Value ({abs(dfRebalanced['Offset'][i]):.3%})
'''
        )

[Portfolio Value]
- 12.295.848,806

[Renda Fixa]
- Sell 348.752,492 of Porfolio Value (2.836%)

[Retorno Absoluto]
- Sell 2.574.217,477 of Porfolio Value (20.936%)

[Renda Variável]
- Buy 294.678,257 of Porfolio Value (2.397%)

[Private Equity]
- Buy 3.135.441,446 of Porfolio Value (25.500%)

[Real Estate]
- Buy 673.860,213 of Porfolio Value (5.480%)

[Commodities]
- Skip 0,000 of Porfolio Value (0.000%)



In [29]:
'''
Código:

1. Tracking Error
2. Filtro dos NaNs
3. Nova Carteira (KING)
4. Melhorar o Filtro
5. Sell/Buy Otimizado (Monte Carlo) | Leônidas
6. Integração com API
t
'''

'\nCódigo:\n\n1. Tracking Error\n2. Filtro dos NaNs\n3. Nova Carteira (KING)\n4. Melhorar o Filtro\n5. Sell/Buy Otimizado (Monte Carlo) | Leônidas\n6. Integração com API\nt\n'

In [30]:
'''
Relatório:


'''

'\nRelatório:\n\n\n'